# Analysing a Simple Circuit using Paricle Swarm Optimzation    ==============================

In this example, we are going to use the `pyswarms` library to analyse a simple circuit by treating it as an optimization problem.The goal of the analysis of the circuit is to find out the current in the circuit.We will use the `pyswarms` library to find an *optimal* solution from a set of candidate solutions.

In [3]:
import sys
# Change directory to access the pyswarms module
sys.path.append('../')

In [4]:
print('Running on Python version: {}'.format(sys.version))

Running on Python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]


In [5]:
# Import modules
import numpy as np

# Import PySwarms
import pyswarms as ps
from pyswarms.backend.topology import Pyramid

import math

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Introduction

We will be analysing a Diode-Resistor Circuit in this tutorial.This type of circuit ensures that current would only flow in the direction of the diode's arrow through the resistor(and any load placed in parallel with the resistor).The diode only lets current to flow in one direction through it, namely the direction that it's triangle points. 

PSO algorithm has been used in various fields of electrical engineering for the aim of optimizing such as design of logical circuits, analog and filter design, integrated circuit design, Microwave circuit design etc.


# Determination Of Cost Function                                                                                              
![alt text](https://user-images.githubusercontent.com/39431903/43938822-29aaf9b8-9c66-11e8-8e54-01530db005c6.png "CIRCUIT")

We have used Shockley equation for this tutorial;

<a href="https://www.codecogs.com/eqnedit.php?latex=I&space;=&space;I_s&space;e^{\frac{v_D}{v_T}}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?I&space;=&space;I_s&space;e^{\frac{v_D}{v_T}}" title="I = I_s e^{\frac{v_D}{v_T}}" /></a>

We will simplify the previous equation to get value of diode voltage:

<img src="https://latex.codecogs.com/gif.latex?v_D&space;=&space;v_T&space;\log{\left&space;|\frac{I}{I_s}\right&space;|}" title="v_D = v_T \log{\left |\frac{I}{I_s}\right |}" />

Using the Kirchoff voltage law we get:

![alt text](https://latex.codecogs.com/gif.latex?U&space;=&space;v_D&space;&plus;&space;v_R)

Now we will restructure this equation to get the value of our cost function for the optimization:

![alt text](https://latex.codecogs.com/gif.latex?c&space;=&space;\left&space;|U-v_D-v_R\right&space;|)

The absolute value is necessary because we don't want any negative currents.

![alt text](https://latex.codecogs.com/gif.latex?c&space;=&space;\left&space;|&space;U&space;-&space;v_T&space;\log{\left&space;|&space;\frac{I}{I_s}&space;\right&space;|}&space;-&space;RI&space;\right&space;|)

Hence, diode current is the parameter we want to optimise, 
For this tutorial we will take these values for our parameters:

. U : 10V

. Is : 9.4 pA

. R : 100 Ohm

. Vt : 25.3 mV

Hence, after substituting the values of parameters and  further simplification the cost function will be reduced to: 

![alt text](https://latex.codecogs.com/gif.latex?c&space;=&space;\left&space;|&space;9.24&space;-&space;0.069\left&space;log{\left&space;|&space;I&space;\right&space;|}&space;-&space;100I&space;\right|)
    

# Initialising The Swarm

The main idea for PSO is that we set a swarm $\mathbf{S}$ composed of particles $\mathbf{P}_n$ into a search space in order to find the optimal solution. The movement of the swarm depends on the cognitive ($c_1$) and social ($c_2$) of all the particles. The cognitive component speaks of the particle's bias towards its personal best from its past experience (i.e., how attracted it is to its own best position). The social component controls how the particles are attracted to the best score found by the swarm (i.e., the global best). High $c_1$ paired with low $c_2$ values can often cause the swarm to stagnate. The inverse can cause the swarm to converge too fast, resulting in suboptimal solutions.

We define our particle $\mathbf{P}$ as:

$$\mathbf{P}\,:=\,\mathbf{X}$$

And the swarm as being composed of $N$ particles with certain positions at a timestep $t$:

$$\mathbf{S}_t\,:=\,[\,\mathbf{P}_1\quad\mathbf{P}_2\quad ... \quad\mathbf{P}_N\,]$$

In this implementation, we designate $\mathbf{P}_1$ as the initial configuration of the manipulator at the zero-position. This means that the angles are equal to 0 and the link offset is also zero. We then generate the $N-1$ particles using a uniform distribution which is controlled by the hyperparameter $\epsilon$.

# Finding The Global Optimum

In order to find the global optimum, the swarm must be moved. This movement is then translated by an update of the current position given the swarm's velocity $\mathbf{V}$. That is:

$$\mathbf{S}_{t+1} = \mathbf{S}_t + \mathbf{V}_{t+1}$$

The velocity is then computed as follows:

$$\mathbf{V}_{t+1} = w\mathbf{V}_t + c_1 r_1 (\mathbf{p}_{best} - \mathbf{p}) + c_2 r_2(\mathbf{g}_{best} - \mathbf{p})$$

Where $r_1$ and $r_2$ denote random values in the intervall $[0,1]$, $\mathbf{p}_{best}$ is the best and $\mathbf{p}$ is the current personal position and $\mathbf{g}_{best}$ is the best position of all the particles. Moreover, $w$ is the inertia weight that controls the "memory" of the swarm's previous position.

# Preparation

Let us now see how this works with the `pyswarms` library. We use the following cost function for its optimisation at different values of diode current: 

In [6]:
%%time

def cost_func(I):
    """Method for definition of our cost function
        Inputs:
        --------
        I : current flowing in the diode
        
        Returns:
        ---------
        cost: our cost function
    """
    cost = abs(9.24 - (0.069 * (math.log(I, 10))) - (100 * I))
    return cost

def cost(cost_func):
    """A decorator for the cost function
    This decorator allows the creation of much simpler cost functions. Instead of
    writing a cost function that returns a shape of :code:`(n_particles, 0)` it enables
    the usage of shorter and simpler cost functions that directly return the cost.
    A simple example might be:
     .. code-block:: python
         import pyswarms
         import numpy as np
         @pyswarms.cost
         def cost_func(x):
            cost = np.abs(np.sum(x))
            return cost
    The decorator expects your cost function to use a d-dimensional array (where
    d is the number of dimensions for the optimization) as and argument.
    .. note::
        Some :code:`numpy` functions return a :code:`np.ndarray` with single values in it.
        Be aware of the fact that without unpacking the value the optimizer will raise
        an exception.
    Parameters
    ----------
    cost_func : callable
        A callable object that can be used as cost function in the optimization
        (must return a :code:`float` or an :code:`int`).
    Returns
    -------
    cost_dec : callable
        The vectorized output for all particles as defined by :code:`cost_func`
    """
    def cost_dec(particles, **kwargs):
        n_particles = particles.shape[0]
        vector = np.array([cost_func(particles[i], **kwargs) for i in range(n_particles)])
        assert vector.shape == (n_particles, ), "The cost function should return a single value."
        return vector
    return cost_dec

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 13.8 µs


# Running The Algorithm
Braced with these preparations we can finally start using the algorithm:

In [7]:
%%time

# Set-up hyperparameters and topology
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
my_topology = Pyramid(static=False)

# Call instance of GlobalBestPSO
optimizer = ps.single.GeneralOptimizerPSO(n_particles=10, dimensions=2,
                                    options=options, topology=my_topology)

# Perform optimization
stats = optimizer.optimize(cost, print_step=100, iters=100, verbose=3)

INFO:pyswarms.single.general_optimizer:Arguments Passed to Objective Function: {}


TypeError: '<' not supported between instances of 'function' and 'function'